In [1]:
import numpy as np
from myhdl import *

from src.utils import IQdata, RTLblocks

In [2]:
# CMult test
dWbits = 8
dW = 2**(dWbits - 1)
shOut = 4 # 2nd input has format (8, 4)
N = 100
din1 = 2**dWbits * ((np.random.rand(N) - .5) + 1j*(np.random.rand(N) - .5))
din1 = np.floor(din1.real) + 1j*np.floor(din1.imag)
din2 = 2**dWbits * ((np.random.rand(N) - .5) + 1j*(np.random.rand(N) - .5))
din2 = np.floor(din2.real) + 1j*np.floor(din2.imag)
din2 /= 2**shOut
print(np.max(din1.real))
print(np.min(din1.real))
print(np.max(din2.imag))
print(np.min(din2.imag))
res = din1 * din2
res = np.floor(res.real) + 1j* np.floor(res.imag)

delayRTL = 4

127.0
-128.0
7.8125
-8.0


In [3]:
rtlOut = np.array([], dtype=complex)

@block
def testbench(autoCheck=True):
    period = 10
    
    i_clk = Signal(bool(0))
    i_rst = ResetSignal(0, active=bool(1), isasync=False)
    i_d1 = IQdata(dWbits)
    i_d2 = IQdata(dWbits)
    i_dv = Signal(bool(0))
    o_d = IQdata(2*dWbits - shOut)
    o_dv = Signal(bool(0))

    dut = RTLblocks.CMult(i_clk, i_rst, i_d1, i_d2, i_dv, o_d, o_dv, shOut)
    
    @always(delay(int(period // 2)))
    def clk_gen():
        i_clk.next = not i_clk
        
    @instance
    def stim():
        yield i_clk.posedge
        i_rst.next = bool(1)
        for _ in range(3):
            yield i_clk.posedge
        i_rst.next = bool(0)
        yield i_clk.negedge
        nTicks = 0
        refPnt = 0
        global rtlOut
        i_dv.next = bool(1)
        for d1, d2 in zip(din1, din2):
            i_d1.I.next = int(d1.real)
            i_d1.Q.next = int(d1.imag)
            i_d2.I.next = int(d2.real * 2**shOut)
            i_d2.Q.next = int(d2.imag * 2**shOut)
            if o_dv:
                rtlOut = np.append(rtlOut, int(o_d.I.val) + 1j*int(o_d.Q.val))
                if nTicks < delayRTL-1:
                    nTicks += 1
                else:
                    if autoCheck:
                        assert (int(o_d.I.val) == res[refPnt].real) and \
                            (int(o_d.Q.val) == res[refPnt].imag), (f"Reference and output " +
                                f"mismatch: {int(o_d.I.val)} + 1j*{int(o_d.Q.val)} != {res[refPnt]}")
                        refPnt += 1
            yield i_clk.negedge
        i_dv.next = bool(0)
        print(' **** Test passed **** ')
        raise StopSimulation
                
    return instances()

tb = testbench(autoCheck=True)
tb.config_sim(trace=False)
tb.run_sim()

 **** Test passed **** 


In [ ]:
i_clk = Signal(bool(0))
i_rst = ResetSignal(0, active=bool(1), isasync=False)
i_d1 = IQdata(dWbits)
i_d2 = IQdata(dWbits)
i_dv = Signal(bool(0))
o_d = IQdata(2*dWbits - shOut)
o_dv = Signal(bool(0))

inst = RTLblocks.CMult(i_clk, i_rst, i_d1, i_d2, i_dv, o_d, o_dv, shOut)
inst.convert(hdl='VHDL', path='./vhdl');

with open('./vhdl/CMult.vhd') as f:
    print(f.read())

-- File: ./vhdl\CMult.vhd
-- Generated by MyHDL 0.11
-- Date: Thu Jun 13 16:56:27 2019


library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;
use std.textio.all;

use work.pck_myhdl_011.all;

entity CMult is
    port (
        i_clk: in std_logic;
        i_rst: in std_logic;
        i_dv: in std_logic;
        o_dv: out std_logic;
        i_d1_I: in signed (7 downto 0);
        i_d1_Q: in signed (7 downto 0);
        i_d2_I: in signed (7 downto 0);
        i_d2_Q: in signed (7 downto 0);
        o_d_I: out signed (11 downto 0);
        o_d_Q: out signed (11 downto 0)
    );
end entity CMult;
-- Complex multiplication block using Karatsuba algorithm (consume three multipliers and 5 adders)
-- ~~~~ input ports ~~~~
-- :param i_clk: clock;
-- :param i_rst: synchronous reset;
-- :param i_d1: input data 1, IQdata();
-- :param i_d2: input data 2, IQdata();
-- :param i_dv: input data valid flag;
-- ~~~~ output ports ~~~~
-- :param o_d: output data, IQdata();
-- :param o_dv: o